# MARA - IMDB_mlh dataset tests - by Bartosz Trojan
The implementation will be based on the official MARA paper
Right now I don't have much to show, but this notebook will be updated

## Imports and data preprocessing

In [1]:
# os.environ['TORCH'] = torch.__version__
# print(torch.__version__)

# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [2]:
import os
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv
from utils.read_data_new import IMDB_mlh
from config import config

imdb = IMDB_mlh()
imdb.info()

/home/lewy700/Documents/MARA/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


IMDB movie type dataset:
 Number of nodes: 5614
 Number of edges: 14715
 Number of edges: layer1: 5443, layer2: 3658, cross_layer: 5614
 Number of features: 1000
 Number of classes: 3
 Number of nodes per class: tensor([ 640, 2438, 2536])


## Model architecture

In [3]:
class DropEdge(nn.Module):
    def __init__(self, simplification_type="l-b-l", p=0.2):
        super().__init__()
        self.simplification_type = simplification_type
        self.p = p

    def forward(self, edges, layers_lengths):
        if(self.simplification_type == "l-b-l"):
            intra_layers_length = torch.sum(layers_lengths[:-1])
            intra_mask = torch.rand(intra_layers_length) > self.p

            intra_layers = edges[:,:intra_layers_length]
            edges = torch.cat([intra_layers[:,intra_mask], edges[:,intra_layers_length:]], dim=1)

            new_layers_lenghts = []
            temp = 0
            for i in range(len(layers_lengths)-1):
                new_layers_lenghts.append(torch.sum(intra_mask[temp:temp + layers_lengths[i]]))
                temp += layers_lengths[i]
            new_layers_lenghts.append(layers_lengths[-1])

            return edges, torch.tensor(new_layers_lenghts)
        
        if(self.simplification_type == "multilayer"):
            mask = torch.rand(edges.shape[1]) > self.p
            edges = edges[:, mask]

            new_layers_lenghts = []
            temp = 0
            for i in range(len(layers_lengths)):
                new_layers_lenghts.append(torch.sum(mask[temp:temp + layers_lengths[i]]))
                temp += layers_lengths[i]

            return edges, torch.tensor(new_layers_lenghts)

In [4]:
# Dlaczego oni nie wspominają o żadnych funkcjach aktywacji w MARZE???

class MARA(nn.Module):
    def __init__(self, simplificaton_type=config["simplification_type"], simplification_stages=config["simplification_stages"], simplification_strategy=config["simplification_strategy"], DE_p=config["DE_p"], NS_k=config["NS_k"]):
        super().__init__()
        torch.manual_seed(1234)
        
        self.simplification_type = simplificaton_type
        self.simplification_stages = simplification_stages
        self.simplification_strategy = simplification_strategy
        self.DE_p = DE_p
        self.NS_k = NS_k
        
        self.conv1 = GCNConv(imdb.get_number_of_features(), 512)
        self.conv2 = GCNConv(512, 256)
        self.conv3 = GCNConv(256, 52)
        self.classifier = nn.Linear(52, imdb.get_number_of_classes())

        self.dropedge = DropEdge(self.simplification_type, self.DE_p)

    def forward(self, x, edges, layers_lengths):
        if self.simplification_stages == "once":
            edges, layers_lengths = self.dropedge(edges, layers_lengths)
            h = self.conv1(x, edges)
            h = h.tanh()
            h = self.conv2(h, edges)
            h = h.tanh()
            h = self.conv3(h, edges)
            h = h.tanh()

        if self.simplification_stages == "each":
            edges, layers_lengths = self.dropedge(edges, layers_lengths)
            h = self.conv1(x, edges)
            h = h.tanh()
            edges, layers_lengths = self.dropedge(edges, layers_lengths)
            h = self.conv2(h, edges)
            h = h.tanh()
            edges, layers_lengths = self.dropedge(edges, layers_lengths)
            h = self.conv3(h, edges)
            h = h.tanh()

        out = torch.sigmoid(self.classifier(h))

        return out, h

model = MARA()
print(model)

MARA(
  (conv1): GCNConv(1000, 512)
  (conv2): GCNConv(512, 256)
  (conv3): GCNConv(256, 52)
  (classifier): Linear(in_features=52, out_features=3, bias=True)
  (dropedge): DropEdge()
)


## Simple model training

In [5]:
model = MARA(simplification_stages="each", simplification_strategy="l-b-l", DE_p=0.2)

out, h = model(imdb.node_features, torch.cat([imdb.layer_1, imdb.layer_2, imdb.cross_edges], dim=0).t(), torch.tensor([imdb.layer_1.shape[0], imdb.layer_2.shape[0], imdb.cross_edges.shape[0]]))

print(out.shape)
print(h.shape)

torch.Size([5614, 3])
torch.Size([5614, 52])


In [6]:
# class DropEdge(torch.autograd.Function):
#     def __init__(self, simplification_type="l-b-l", p=0.2):
#         self.simplification_type = simplification_type
#         self.p = p
        
#     @staticmethod
#     def forward(ctx, intra_edges, cross_edges):
#         if(ctx.simplification_type == "l-b-l"):
#             mask = torch.rand(intra_edges.shape) > ctx.p
#             ctx.save_for_backward(mask)

#             return intra_edges[mask], cross_edges
        
#         if(ctx.simplification_type == "multilayer"):
#             intra_mask = torch.rand(intra_edges.shape) > ctx.p
#             cross_mask = torch.rand(cross_edges.shape) > ctx.p
#             ctx.save_for_backward(intra_mask, cross_mask)

#             return intra_edges[intra_mask], cross_edges[cross_mask]

#     @staticmethod
#     def backward(ctx, grad_output):
#         if(ctx.simplification_type == "l-b-l"):
#             mask = ctx.saved_tensors
            
#             return intra_edges[mask], cross_edges
        
#         if(ctx.simplification_type == "multilayer"):
#             intra_mask = torch.rand(intra_edges.shape) > ctx.p
#             cross_mask = torch.rand(cross_edges.shape) > ctx.p
#             ctx.save_for_backward(intra_mask, cross_mask)

#             return intra_edges[intra_mask], cross_edges[cross_mask]

#         A = grad_output * D
#         return A / (1-p)

In [7]:
model = MARA()
criterion = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) 

def accuracy(preds, labels):
    predicted_labels = torch.argmax(preds, dim=1)
    accuracy = (predicted_labels == labels).float().mean()

    return accuracy

train_mask = imdb.get_training_mask(mask_size=0.25)
print(f"training mask: {torch.sum(train_mask)}/{imdb.get_number_of_nodes()}")

def train(data):
    optimizer.zero_grad()
    edges = torch.cat([data.layer_1, data.layer_2, data.cross_edges], dim=0).t()
    layers_lengths = torch.tensor([data.layer_1.shape[0], data.layer_2.shape[0], data.cross_edges.shape[0]], dtype=torch.int64)
    out, h = model(data.node_features, edges, layers_lengths) 

    test_acc = accuracy(out[train_mask == False], imdb.classes[train_mask == False])

    loss = criterion(out[train_mask], data.classes[train_mask])
    acc = accuracy(out[train_mask], data.classes[train_mask])

    loss.backward()
    optimizer.step()

    return loss, acc, test_acc

for epoch in range(201):
    loss, acc, test_acc = train(imdb)
    if (epoch+1)%10 == 0:
        print("======== ",epoch+1," ========")
        print(f"Loss: {loss}")
        print(f"Train accuracy: {acc}")
        print(f"Test accuracy: {test_acc}")
data = imdb
edges = torch.cat([data.layer_1, data.layer_2, data.cross_edges], dim=0).t()
layers_lengths = torch.tensor([data.layer_1.shape[0], data.layer_2.shape[0], data.cross_edges.shape[0]], dtype=torch.int64)
out, h = model(data.node_features, edges, layers_lengths) 

print(f"Final accuracy - whole dataset: {accuracy(out, data.classes)}, test_set: {accuracy(out[train_mask == False], data.classes[train_mask == False])}")

training mask: 1388/5614


========  10  ========
Loss: 0.7244137525558472
Train accuracy: 0.8386167287826538
Test accuracy: 0.7084713578224182
========  20  ========
Loss: 0.6256277561187744
Train accuracy: 0.9056196212768555
Test accuracy: 0.712257444858551
========  30  ========
Loss: 0.6149313449859619
Train accuracy: 0.960374653339386
Test accuracy: 0.6997160315513611
========  40  ========
Loss: 0.5956122279167175
Train accuracy: 0.9668588042259216
Test accuracy: 0.6876478791236877
========  50  ========
Loss: 0.5858041048049927
Train accuracy: 0.9711815714836121
Test accuracy: 0.693327009677887
========  60  ========
Loss: 0.5789570212364197
Train accuracy: 0.9776657223701477
Test accuracy: 0.6836251616477966
========  70  ========
Loss: 0.5722885131835938
Train accuracy: 0.9812679886817932
Test accuracy: 0.6798390746116638
========  80  ========
Loss: 0.5712034702301025
Train accuracy: 0.9827089309692383
Test accuracy: 0.671320378780365
========  90  ========
Loss: 0.5694078803062439
Train accuracy: 0.98

KeyboardInterrupt: 